# Student Performance Prediction (Classification)

**📊 Dataset:** `student_performance.csv`  
**📚 Source:** [Kaggle – student_performance Dataset](https://www.kaggle.com/)  




## 🎯 Goal
The goal of this project is to predict student academic performance using **supervised machine learning (classification) with Apache Spark**.  
By analyzing students’ demographic, behavioral, and academic features, the model aims to classify students based on their expected performance level, helping identify students who may need early academic support and improve educational decision-making.




## 📈 Description
The dataset includes features such as:  
- `StudentID`, `Gender`, `AttendanceRate`, `StudyHoursPerWeek`  
- `PreviousGrade`, `ExtracurricularActivities`, `ParentalSupport`, `OnlineClassesTaken`  
- `FinalGrade` (Target Variable)  

This project predicts students’ academic performance levels to support **data-driven educational decisions**.




In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.ml.feature import StringIndexer, OneHotEncoder
import pyspark.sql.functions as F

In [2]:
# Create Spark Session
spark = SparkSession.builder \
    .appName("Student Performance - Data Cleaning") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

print("✅ Spark Session Created Successfully")

✅ Spark Session Created Successfully


# Phase 1: Data Overview & Understanding

In [3]:
df = spark.read.csv(
    "../data/raw/student_performance_updated_1000.csv",
    header=True,
    inferSchema=True
)

print("✅ Dataset Loaded Successfully")

✅ Dataset Loaded Successfully


In [4]:
# Dataset Shape
rows = df.count()
cols = len(df.columns)

print(f"📊 Dataset Shape: {rows} rows, {cols} columns")

📊 Dataset Shape: 1000 rows, 12 columns


In [5]:
# Preview Dataset
print("🔹 First 5 rows:")
df.show(5)

🔹 First 5 rows:
+---------+-------+------+--------------+-----------------+-------------+-------------------------+---------------+----------+-----------+--------------+--------------------+
|StudentID|   Name|Gender|AttendanceRate|StudyHoursPerWeek|PreviousGrade|ExtracurricularActivities|ParentalSupport|FinalGrade|Study Hours|Attendance (%)|Online Classes Taken|
+---------+-------+------+--------------+-----------------+-------------+-------------------------+---------------+----------+-----------+--------------+--------------------+
|      1.0|   John|  Male|          85.0|             15.0|         78.0|                      1.0|           High|      80.0|        4.8|          59.0|               false|
|      2.0|  Sarah|Female|          90.0|             20.0|         85.0|                      2.0|         Medium|      87.0|        2.2|          70.0|                true|
|      3.0|   Alex|  Male|          78.0|             10.0|         65.0|                      0.0|          

In [6]:
# Preview Dataset
print("🔹 Random sample:")
df.sample(fraction=0.01).show(5)

🔹 Random sample:
+---------+----------------+------+--------------+-----------------+-------------+-------------------------+---------------+----------+-----------+--------------+--------------------+
|StudentID|            Name|Gender|AttendanceRate|StudyHoursPerWeek|PreviousGrade|ExtracurricularActivities|ParentalSupport|FinalGrade|Study Hours|Attendance (%)|Online Classes Taken|
+---------+----------------+------+--------------+-----------------+-------------+-------------------------+---------------+----------+-----------+--------------+--------------------+
|   1949.0|  Jessica Ortega|  Male|          91.0|             20.0|         null|                      1.0|           High|      72.0|        3.8|          58.0|               false|
|   6091.0|     Morgan Bell|  Male|          82.0|             null|         85.0|                     null|           High|      78.0|        4.7|          91.0|                true|
|   2501.0|  Melissa Barber|  Male|          91.0|             

In [7]:
# Dataset Schema & Info
print("🔹 Dataset Schema:")
df.printSchema()

🔹 Dataset Schema:
root
 |-- StudentID: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- AttendanceRate: double (nullable = true)
 |-- StudyHoursPerWeek: double (nullable = true)
 |-- PreviousGrade: double (nullable = true)
 |-- ExtracurricularActivities: double (nullable = true)
 |-- ParentalSupport: string (nullable = true)
 |-- FinalGrade: double (nullable = true)
 |-- Study Hours: double (nullable = true)
 |-- Attendance (%): double (nullable = true)
 |-- Online Classes Taken: boolean (nullable = true)



#### Identify Columns Types

In [8]:
# Identify Numerical Columns
numerical_cols = [
    field.name for field in df.schema.fields
    if isinstance(field.dataType, (IntegerType, DoubleType))
]

print("📌 Numerical Columns:", numerical_cols)

📌 Numerical Columns: ['StudentID', 'AttendanceRate', 'StudyHoursPerWeek', 'PreviousGrade', 'ExtracurricularActivities', 'FinalGrade', 'Study Hours', 'Attendance (%)']


In [9]:
# Identify Categorical Columns
categorical_cols = [
    field.name for field in df.schema.fields
    if field.name not in numerical_cols
]
print("📌 Categorical Columns:", categorical_cols)

📌 Categorical Columns: ['Name', 'Gender', 'ParentalSupport', 'Online Classes Taken']


In [10]:
# Unique Values per Column
print("🔹 Unique values per column:")
for col_name in df.columns:
    print(f"{col_name}: {df.select(col_name).distinct().count()}")

🔹 Unique values per column:
StudentID: 917
Name: 963
Gender: 3
AttendanceRate: 10
StudyHoursPerWeek: 11
PreviousGrade: 11
ExtracurricularActivities: 5
ParentalSupport: 4
FinalGrade: 11
Study Hours: 53
Attendance (%): 53
Online Classes Taken: 3


# Data Cleaning

In [11]:
# Check Missing Values
print("🔹 Missing values per column:")
df.select([
    F.count(F.when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()

🔹 Missing values per column:
+---------+----+------+--------------+-----------------+-------------+-------------------------+---------------+----------+-----------+--------------+--------------------+
|StudentID|Name|Gender|AttendanceRate|StudyHoursPerWeek|PreviousGrade|ExtracurricularActivities|ParentalSupport|FinalGrade|Study Hours|Attendance (%)|Online Classes Taken|
+---------+----+------+--------------+-----------------+-------------+-------------------------+---------------+----------+-----------+--------------+--------------------+
|       40|  34|    48|            40|               50|           33|                       43|             22|        40|         24|            41|                  25|
+---------+----+------+--------------+-----------------+-------------+-------------------------+---------------+----------+-----------+--------------+--------------------+



Handle Missing Values
Strategy:
- Numerical → Median (avoid skewing)
- Categorical → Mode (most frequent value)

In [12]:
# Handle Numerical Missing Values (Median)
for col_name in numerical_cols:
    median_value = df.approxQuantile(col_name, [0.5], 0.01)[0]
    df = df.fillna({col_name: median_value})


In [13]:
# Handle Categorical Missing Values (Mode)

# Select categorical columns
categorical_cols = [col for col, dtype in df.dtypes if dtype == 'string']

for col_name in categorical_cols:
    # Get the most frequent value (mode) for the column
    mode_row = df.groupBy(col_name).count().orderBy(F.desc("count")).first()

    # If mode exists and is not None, fill missing values with it
    if mode_row is not None and mode_row[0] is not None:
        mode_value = str(mode_row[0])  # Ensure it's a string
        df = df.na.fill({col_name: mode_value})
    else:
        print(f"⚠️ Column {col_name} is empty or all null, skipping fillna.")

print("✅ Missing values for categorical columns handled successfully")

⚠️ Column Name is empty or all null, skipping fillna.
✅ Missing values for categorical columns handled successfully


In [14]:
# Validate Missing Values Removal
print("🔹 Missing values after cleaning:")
df.select([
    F.count(F.when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()

🔹 Missing values after cleaning:
+---------+----+------+--------------+-----------------+-------------+-------------------------+---------------+----------+-----------+--------------+--------------------+
|StudentID|Name|Gender|AttendanceRate|StudyHoursPerWeek|PreviousGrade|ExtracurricularActivities|ParentalSupport|FinalGrade|Study Hours|Attendance (%)|Online Classes Taken|
+---------+----+------+--------------+-----------------+-------------+-------------------------+---------------+----------+-----------+--------------+--------------------+
|        0|  34|     0|             0|                0|            0|                        0|              0|         0|          0|             0|                  25|
+---------+----+------+--------------+-----------------+-------------+-------------------------+---------------+----------+-----------+--------------+--------------------+



In [15]:
# Check Duplicates
duplicates_count = df.count() - df.dropDuplicates().count()
print(f"🔹 Number of duplicate rows: {duplicates_count}")

# Remove Duplicates
df = df.dropDuplicates()
print("✅ Duplicate rows removed")

print("📊 New Dataset Shape:", df.count(), "rows")

🔹 Number of duplicate rows: 0
✅ Duplicate rows removed
📊 New Dataset Shape: 1000 rows


In [16]:
# Detect Outliers using IQR (Numerical Columns)
for col_name in numerical_cols:
    Q1 = df.approxQuantile(col_name, [0.25], 0.01)[0]
    Q3 = df.approxQuantile(col_name, [0.75], 0.01)[0]
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    df = df.withColumn(
        col_name,
        when(col(col_name) < lower, lower)
        .when(col(col_name) > upper, upper)
        .otherwise(col(col_name))
    )

print("✅ Outliers handled using IQR capping")

✅ Outliers handled using IQR capping


In [17]:
# Create target variable for classification
df = df.withColumn(
    "PerformanceLevel",
    when(col("FinalGrade") >= 85, "High")
    .when(col("FinalGrade") >= 70, "Medium")
    .otherwise("Low")
)

print("✅ Target variable (PerformanceLevel) created")

# NOTE:
# PerformanceLevel will be used as the target variable
# in the ML modeling phase (model.py)


✅ Target variable (PerformanceLevel) created


In [18]:
#  Final Cleaned Dataset Validation
print("📊 Final Dataset Shape:")
print("Rows:", df.count())
print("Columns:", len(df.columns))

print("🔹 Final Schema:")
df.printSchema()

df.describe().show()

📊 Final Dataset Shape:
Rows: 1000
Columns: 13
🔹 Final Schema:
root
 |-- StudentID: double (nullable = false)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = false)
 |-- AttendanceRate: double (nullable = false)
 |-- StudyHoursPerWeek: double (nullable = false)
 |-- PreviousGrade: double (nullable = false)
 |-- ExtracurricularActivities: double (nullable = false)
 |-- ParentalSupport: string (nullable = false)
 |-- FinalGrade: double (nullable = false)
 |-- Study Hours: double (nullable = false)
 |-- Attendance (%): double (nullable = false)
 |-- Online Classes Taken: boolean (nullable = true)
 |-- PerformanceLevel: string (nullable = false)

+-------+------------------+--------------+------+-----------------+-----------------+-----------------+-------------------------+---------------+-----------------+------------------+------------------+----------------+
|summary|         StudentID|          Name|Gender|   AttendanceRate|StudyHoursPerWeek|    PreviousGrade|Extrac

### Save cleaned dataset


In [19]:
df.toPandas().to_csv(
    "../data/cleaned/student_performance_cleaned.csv",
    index=False
)

print("✅ Cleaned dataset saved successfully")

✅ Cleaned dataset saved successfully


## Statistical Analysis & Data Summaries



In [20]:
# Select numerical columns
numeric_cols = [c for c, t in df.dtypes if t in ('int', 'double')]

# Summary statistics
df.select(numeric_cols).describe().show()


+-------+------------------+-----------------+-----------------+-----------------+-------------------------+-----------------+------------------+------------------+
|summary|         StudentID|   AttendanceRate|StudyHoursPerWeek|    PreviousGrade|ExtracurricularActivities|       FinalGrade|       Study Hours|    Attendance (%)|
+-------+------------------+-----------------+-----------------+-----------------+-------------------------+-----------------+------------------+------------------+
|  count|              1000|             1000|             1000|             1000|                     1000|             1000|              1000|              1000|
|   mean|          5412.859|            85.61|           17.599|           77.612|                    1.498|           80.029|2.4337000000000018|            76.437|
| stddev|2600.1236459239967|7.200398999065666| 6.11470265741422|9.840238121421967|       1.0291040059464616|9.301649298897463|1.5028199419018158|15.086007378316456|
|    min| 

### Categorical Feature Distributions

In [21]:
# Frequency counts for categorical features
for col_name in categorical_cols:
    print(f"\nDistribution of {col_name}:")
    df.groupBy(col_name).count().orderBy("count", ascending=False).show()



Distribution of Name:
+------------------+-----+
|              Name|count|
+------------------+-----+
|              null|   34|
|       Andrea Frey|    2|
|     Anthony Smith|    2|
|      Erica Miller|    2|
| Kimberly Harrison|    2|
|Jessica Richardson|    1|
|   Lonnie Williams|    1|
|  Samantha Mendoza|    1|
|    Alyssa Schmidt|    1|
|     Michael Smith|    1|
|       Gina Palmer|    1|
|       Joseph Holt|    1|
| Christina Johnson|    1|
|  Elizabeth Chavez|    1|
|       Sarah Young|    1|
|      Heather Hill|    1|
|     Candace Kelly|    1|
|    Mrs. Jill Long|    1|
|     Elijah Wagner|    1|
|  Courtney Bradley|    1|
+------------------+-----+
only showing top 20 rows


Distribution of Gender:
+------+-----+
|Gender|count|
+------+-----+
|  Male|  549|
|Female|  451|
+------+-----+


Distribution of ParentalSupport:
+---------------+-----+
|ParentalSupport|count|
+---------------+-----+
|           High|  367|
|         Medium|  328|
|            Low|  305|
+--------

### Correlation Analysis

In [22]:
# Convert numerical features to Pandas for correlation analysis
corr_matrix = df.select(numeric_cols).toPandas().corr()

corr_matrix.round(2)


,StudentID,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,FinalGrade,Study Hours,Attendance (%)
StudentID,1.00,0.05,-0.01,-0.03,-0.04,0.08,0.04,-0.01
AttendanceRate,0.05,1.00,0.01,0.03,-0.02,-0.01,-0.02,0.02
StudyHoursPerWeek,-0.01,0.01,1.00,-0.01,0.03,0.03,-0.02,0.06
PreviousGrade,-0.03,0.03,-0.01,1.00,0.06,0.00,-0.04,0.05
ExtracurricularActivities,-0.04,-0.02,0.03,0.06,1.00,-0.03,-0.04,-0.01
FinalGrade,0.08,-0.01,0.03,0.00,-0.03,1.00,0.04,0.04
Study Hours,0.04,-0.02,-0.02,-0.04,-0.04,0.04,1.00,-0.11
Attendance (%),-0.01,0.02,0.06,0.05,-0.01,0.04,-0.11,1.00


### Multicollinearity Assessment

In [23]:
high_corr_pairs = []

for i in range(len(numeric_cols)):
    for j in range(i + 1, len(numeric_cols)):
        corr_value = corr_matrix.iloc[i, j]
        if abs(corr_value) > 0.8:
            high_corr_pairs.append(
                (numeric_cols[i], numeric_cols[j], round(corr_value, 2))
            )

high_corr_pairs


[]

### Summary Tables


In [24]:
from pyspark.sql.functions import countDistinct

# Number of unique categories per categorical feature
categorical_summary = df.select([
    countDistinct(c).alias(c) for c in categorical_cols
])

categorical_summary.show()


+----+------+---------------+
|Name|Gender|ParentalSupport|
+----+------+---------------+
| 962|     2|              3|
+----+------+---------------+



Preprocessing and ML Modeling

In [25]:
# Additional imports for ML (not imported above)
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Load cleaned dataset
data = spark.read.csv(
    '../data/cleaned/student_performance_cleaned.csv',
    header=True,
    inferSchema=True
)

# Cast 'Online Classes Taken' to IntegerType (0 or 1)
# Note: This column had nulls in raw data, fill any remaining with mode (0)
data = data.withColumn("Online Classes Taken", col("Online Classes Taken").cast(IntegerType()))
data = data.fillna({'Online Classes Taken': 0})

print("✅ Cleaned dataset loaded for ML")
print(f"📊 Shape: {data.count()} rows, {len(data.columns)} columns")
data.printSchema()

✅ Cleaned dataset loaded for ML
📊 Shape: 1000 rows, 13 columns
root
 |-- StudentID: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- AttendanceRate: double (nullable = true)
 |-- StudyHoursPerWeek: double (nullable = true)
 |-- PreviousGrade: double (nullable = true)
 |-- ExtracurricularActivities: double (nullable = true)
 |-- ParentalSupport: string (nullable = true)
 |-- FinalGrade: double (nullable = true)
 |-- Study Hours: double (nullable = true)
 |-- Attendance (%): double (nullable = true)
 |-- Online Classes Taken: integer (nullable = false)
 |-- PerformanceLevel: string (nullable = true)



In [26]:
train_df, test_df = data.randomSplit([0.8, 0.2], seed=42)

print("Train size:", train_df.count())
print("Test size:", test_df.count())

Train size: 838
Test size: 162


In [27]:
label_indexer = StringIndexer(inputCol='PerformanceLevel',outputCol='label').fit(train_df)

train_df = label_indexer.transform(train_df)
test_df = label_indexer.transform(test_df)

In [28]:
categorical_cols = ['Gender','ParentalSupport']

indexers = [StringIndexer(inputCol=c,outputCol=f"{c}_idx",handleInvalid='keep') for c in categorical_cols]

encoders = [OneHotEncoder(inputCol=f"{c}_idx",outputCol=f"{c}_ohe") for c in categorical_cols]

In [29]:
numeric_cols = ['AttendanceRate','StudyHoursPerWeek','PreviousGrade','ExtracurricularActivities','Study Hours','Attendance (%)', 'Online Classes Taken']

numeric_assembler = VectorAssembler(inputCols=numeric_cols,outputCol='numeric_features')

scaler = StandardScaler(inputCol='numeric_features',outputCol='scaled_numeric_features')

In [30]:
feature_cols = ['scaled_numeric_features'] + [f"{c}_ohe" for c in categorical_cols]

final_assembler = VectorAssembler(
inputCols=feature_cols,
outputCol='features'
)

In [31]:
log_reg = LogisticRegression(
featuresCol='features',
labelCol='label'
)

rand_forest = RandomForestClassifier(
featuresCol='features',
labelCol='label',
seed=42
)

In [32]:
lr_param_grid = (ParamGridBuilder()
.addGrid(log_reg.regParam, [0.01, 0.1])
.addGrid(log_reg.elasticNetParam, [0.0, 0.5])
.build()
)

rf_param_grid = (ParamGridBuilder()
.addGrid(rand_forest.numTrees, [50, 100])
.addGrid(rand_forest.maxDepth, [5, 10])
.build()
)

In [33]:
common_stages = (
indexers +
encoders +
[numeric_assembler,
scaler,
final_assembler]
)

lr_pipeline = Pipeline(stages=common_stages + [log_reg])
rf_pipeline = Pipeline(stages=common_stages + [rand_forest])

In [ ]:
f1_evaluator = MulticlassClassificationEvaluator(
labelCol='label',
predictionCol='prediction',
metricName='f1'
)

lr_cv = CrossValidator(
estimator=lr_pipeline,
estimatorParamMaps=lr_param_grid,
evaluator=f1_evaluator,
numFolds=5
)

rf_cv = CrossValidator(
estimator=rf_pipeline,
estimatorParamMaps=rf_param_grid,
evaluator=f1_evaluator,
numFolds=5
)

lr_model = lr_cv.fit(train_df)
rf_model = rf_cv.fit(train_df)

In [ ]:
accuracy_eval = MulticlassClassificationEvaluator(
labelCol='label',
predictionCol='prediction',
metricName='accuracy'
)

f1_evaluator = MulticlassClassificationEvaluator(
labelCol='label',
predictionCol='prediction',
metricName='f1'
)

lr_preds = lr_model.transform(test_df)
rf_preds = rf_model.transform(test_df)

print("Logistic Regression →",
"Accuracy:", accuracy_eval.evaluate(lr_preds),
"F1:", f1_evaluator.evaluate(lr_preds))

print("Random Forest →",
"Accuracy:", accuracy_eval.evaluate(rf_preds),
"F1:", f1_evaluator.evaluate(rf_preds))

Logistic Regression → Accuracy: 0.41358024691358025 F1: 0.3036709115140488
Random Forest → Accuracy: 0.43209876543209874 F1: 0.3767924288111021
